In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
from sklearn.model_selection import train_test_split
import pandas as pd

from IPython import get_ipython # type: ignore
import os

torch.manual_seed(0)

# get the notebook name
ip = get_ipython()
path = None
if '__vsc_ipynb_file__' in ip.user_ns: # type: ignore
    path = ip.user_ns['__vsc_ipynb_file__'] # type: ignore

os.makedirs('models/', exist_ok=True)
model_file_name = f"models/{os.path.basename(path)[:-6]}.pt" # type: ignore

Data

In [2]:
movie_path = 'dataset/ml-latest-small/movies_with_wikidata_values.csv'
rating_path = 'dataset/ml-latest-small/ratings.csv'
directed_by_path = 'dataset/ml-latest-small/directed_by.csv'
stars_path = 'dataset/ml-latest-small/stars.csv'


movies_df = pd.read_csv(movie_path, index_col='movieId')
ratings_df = pd.read_csv(rating_path)[["userId", "movieId", "rating"]]

directed_by_df = pd.read_csv(directed_by_path)[["movieId", "director"]]
stars_df = pd.read_csv(stars_path)[["movieId", "castMember"]]

In [3]:
movie_ids_tensor = torch.LongTensor(range(len(ratings_df['movieId'].unique())))
user_ids_tensor = torch.LongTensor(range(len(ratings_df['userId'].unique())))

director_ids_tensor = torch.LongTensor(range(len(directed_by_df['director'].unique())))
star_ids_tensor = torch.LongTensor(range(len(stars_df['castMember'].unique())))

In [4]:
unique_user_id = ratings_df['userId'].unique()
unique_user_id = pd.DataFrame(data={
    'userId': unique_user_id,
    'mappedUserId': pd.RangeIndex(len(unique_user_id))
    })

# Create a mapping from the movieId to a unique consecutive value in the range [0, num_movies]:
unique_movie_id = ratings_df['movieId'].unique()
unique_movie_id = pd.DataFrame(data={
    'movieId': unique_movie_id,
    'mappedMovieId': pd.RangeIndex(len(unique_movie_id))
    })

unique_director_id = directed_by_df['director'].unique()
unique_director_id = pd.DataFrame(data={
    'director': unique_director_id,
    'mappedDirector': pd.RangeIndex(len(unique_director_id))
    })

unique_star_id = stars_df['castMember'].unique()
unique_star_id = pd.DataFrame(data={
    'castMember': unique_star_id,
    'mappedCastMember': pd.RangeIndex(len(unique_star_id))
    })

# Merge the mappings with the original data frame:
ratings_df = ratings_df.merge(unique_user_id, on='userId')
ratings_df = ratings_df.merge(unique_movie_id, on='movieId')

stars_df = stars_df.merge(unique_movie_id, on='movieId').merge(unique_star_id, on='castMember')
directed_by_df = directed_by_df.merge(unique_movie_id, on='movieId').merge(unique_director_id, on='director')

ratings_df_train, ratings_df_test = train_test_split(ratings_df, test_size=0.2, random_state=42)
ratings_df_test, ratings_df_val = train_test_split(ratings_df_test, test_size=0.5, random_state=42)

In [5]:
def create_data(ratings_df):
    edge_index = torch.stack([
    torch.tensor(ratings_df['mappedUserId'].values),
    torch.tensor(ratings_df['mappedMovieId'].values)]
    , dim=0)

    directed_by_edge_index = torch.stack([
    torch.tensor(directed_by_df['mappedMovieId'].values),
    torch.tensor(directed_by_df['mappedDirector'].values)]
    , dim=0)

    stars_edge_index = torch.stack([
    torch.tensor(stars_df['mappedMovieId'].values),
    torch.tensor(stars_df['mappedCastMember'].values)]
    , dim=0)

    assert edge_index.shape == (2, len(ratings_df))
    data = HeteroData()
    # Add the user nodes:
    data['user'].x = user_ids_tensor # [num_users]
    # Add the movie nodes:
    data['movie'].x = movie_ids_tensor  # [num_movies]
    # Add the directors nodes:
    data['director'].x = director_ids_tensor # [num_directors]
    # Add the stars nodes:
    data['star'].x = star_ids_tensor # [num_stars]

    # Add the rating edges:
    data['user', 'rates', 'movie'].edge_index = edge_index  # [2, num_ratings]
    # Add the rating labels:
    rating = torch.from_numpy(ratings_df['rating'].values).to(torch.float)
    data['user', 'rates', 'movie'].edge_label = rating  # [num_ratings]

    # Add the directed by edges:
    data['movie', 'directed_by', 'director'].edge_index = directed_by_edge_index

    # Add the stars edges:
    data['movie', 'stars', 'star'].edge_index = stars_edge_index

    # We also need to make sure to add the reverse edges from movies to users
    # in order to let a GNN be able to pass messages in both directions.
    # We can leverage the `T.ToUndirected()` transform for this from PyG:
    data = T.ToUndirected()(data)

    # With the above transformation we also got reversed labels for the edges.
    # We are going to remove them:
    del data['movie', 'rev_rates', 'user'].edge_label

    assert data['user'].num_nodes == len(unique_user_id)
    assert data['user', 'rates', 'movie'].num_edges == len(ratings_df)

    return data

In [6]:
train_data, val_data, test_data = create_data(ratings_df_train), create_data(ratings_df_val), create_data(ratings_df_test)

In [13]:
class GraphEmbedding(torch.nn.Module):
    def __init__(self, num_users, user_embedding_dim, num_movies, movie_embedding_dim, num_directors, director_embedding_dim, num_stars, star_embedding_dim):
        super().__init__()
        self.user_embedding = torch.nn.Embedding(num_users, user_embedding_dim)
        self.movie_embedding = torch.nn.Embedding(num_movies, movie_embedding_dim)
        self.director_embedding = torch.nn.Embedding(num_directors, director_embedding_dim)
        self.star_embedding = torch.nn.Embedding(num_stars, star_embedding_dim)

    def forward(self, x_dict):
        x_dict['user'] = self.user_embedding(x_dict['user'])
        x_dict['movie'] = self.movie_embedding(x_dict['movie'])
        x_dict['director'] = self.director_embedding(x_dict['director'])
        x_dict['star'] = self.star_embedding(x_dict['star'])
        return x_dict

class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = torch.nn.Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['user'][row], z_dict['movie'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels, user_dim, movie_dim, director_dim, star_dim):
        super().__init__()
        self.graph_embedding = GraphEmbedding(len(user_ids_tensor), user_dim, len(movie_ids_tensor), movie_dim, len(director_ids_tensor), director_dim, len(star_ids_tensor), star_dim)
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, train_data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        x_dict = self.graph_embedding(x_dict)
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(hidden_channels=32, user_dim=150, movie_dim=400, director_dim=150, star_dim=150).to(device)
print(model)

Model(
  (graph_embedding): GraphEmbedding(
    (user_embedding): Embedding(610, 150)
    (movie_embedding): Embedding(9724, 400)
    (director_embedding): Embedding(4302, 150)
    (star_embedding): Embedding(43410, 150)
  )
  (encoder): GraphModule(
    (conv1): ModuleDict(
      (user__rates__movie): SAGEConv((-1, -1), 32, aggr=mean)
      (movie__directed_by__director): SAGEConv((-1, -1), 32, aggr=mean)
      (movie__stars__star): SAGEConv((-1, -1), 32, aggr=mean)
      (movie__rev_rates__user): SAGEConv((-1, -1), 32, aggr=mean)
      (director__rev_directed_by__movie): SAGEConv((-1, -1), 32, aggr=mean)
      (star__rev_stars__movie): SAGEConv((-1, -1), 32, aggr=mean)
    )
    (conv2): ModuleDict(
      (user__rates__movie): SAGEConv((-1, -1), 32, aggr=mean)
      (movie__directed_by__director): SAGEConv((-1, -1), 32, aggr=mean)
      (movie__stars__star): SAGEConv((-1, -1), 32, aggr=mean)
      (movie__rev_rates__user): SAGEConv((-1, -1), 32, aggr=mean)
      (director__rev_direct

In [14]:
# Initialize optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

# Define training function
def train(model):
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['user', 'movie'].edge_index)
    target = train_data['user', 'movie'].edge_label
    loss = F.mse_loss(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)

# Define test function
@torch.no_grad()
def test(model, data):
    data = data.to(device)
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['user', 'movie'].edge_index)
    pred = pred.clamp(min=0, max=5)
    target = data['user', 'movie'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)

best_val_rmse = float('inf')
best_epoch = 0

# Training loop
for epoch in range(1, 201):
    train_data = train_data.to(device)
    loss = train(model)
    train_rmse = test(model, train_data)
    val_rmse = test(model, val_data)

    # Check if the current validation RMSE is the best
    if val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        torch.save(model.state_dict(), model_file_name)
        print(f'Saving model with val_rmse: {val_rmse:.4f} at epoch {epoch}')

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, Val: {val_rmse:.4f}')

Saving model with val_rmse: 2.9508 at epoch 1
Epoch: 001, Loss: 3.7798, Train: 2.9776, Val: 2.9508
Saving model with val_rmse: 1.9836 at epoch 2
Epoch: 002, Loss: 2.9776, Train: 2.0019, Val: 1.9836
Saving model with val_rmse: 1.2370 at epoch 3
Epoch: 003, Loss: 2.0019, Train: 1.1992, Val: 1.2370
Epoch: 004, Loss: 1.2309, Train: 1.5505, Val: 1.5812
Epoch: 005, Loss: 1.8852, Train: 1.3519, Val: 1.4027
Saving model with val_rmse: 1.0571 at epoch 6
Epoch: 006, Loss: 1.4160, Train: 1.0117, Val: 1.0571
Epoch: 007, Loss: 1.0117, Train: 1.1812, Val: 1.1849
Epoch: 008, Loss: 1.1812, Train: 1.2889, Val: 1.2826
Epoch: 009, Loss: 1.2889, Train: 1.2089, Val: 1.2092
Epoch: 010, Loss: 1.2089, Train: 1.0407, Val: 1.0658
Epoch: 011, Loss: 1.0407, Train: 1.0004, Val: 1.0626
Epoch: 012, Loss: 1.0028, Train: 1.1075, Val: 1.1846
Epoch: 013, Loss: 1.1193, Train: 1.0945, Val: 1.1773
Saving model with val_rmse: 1.0492 at epoch 14
Epoch: 014, Loss: 1.1028, Train: 0.9675, Val: 1.0492
Saving model with val_rmse:

In [12]:
print(f'Loading model with val_rmse: {best_val_rmse:.4f} at epoch {best_epoch}')
model.load_state_dict(torch.load(model_file_name))

print("Test rmse:", test(model, test_data))

Loading model with val_rmse: 0.9171 at epoch 68
Test rmse: 0.906782329082489
